In [36]:
%load_ext autoreload
%autoreload 2
# Enable imports form top-level of project (edit top_level_path accordingly)
import os
import sys
top_level_path = os.path.abspath(os.path.join('..'))
if top_level_path not in sys.path:
	sys.path.append(top_level_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
from pyqubo import Array, LogEncInteger
import neal
from longestpath import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List

In [38]:
# This function turns a graph into new graph with a tail of specified length.
# All vertices in the original graph are connected to the beginning of the tail.
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	# We set the vertex amount of tail_graph and copy the edges of the original graph into tail_graph.edges.
	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	
	#The tail part starts at index graph.vertices.
	#We connect all the original vertices to the beginning of the tail.
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	#We add the connections within the tail.
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


#This functions computes for every vertex in a graph the indices of the edges that are connected to a specific vertex.
#These collections of indices are stored in lists of integers.
#The lists are then stored in an outer list of which the indices correspond to the vertices of the graph.
def make_vertex_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		out[v_1].append(i)
		out[v_2].append(i)
	
	return out

#We endow an orientation on all the edged in a graph.
#Edge (i,j) goes from i to j.
#With respect to this orientation, we compute for each vertex in a graph which edges point towards that vertex
def make_in_flow_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(_, v_2) = graph.edges[i]

		out[v_2].append(i)
	
	return out

#We also compute, for each vertex, which edges point away from that vertex
def make_out_flow_edges(graph : StandardGraph) -> List[List[int]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, _) = graph.edges[i]

		out[v_1].append(i)
	
	return out



In [39]:

#We set a graph of which we want to compute the longest path.
graph = StandardGraph(3, [
	(0,1),
    (1,2),
    (0,2)
])

# graph = gen_planted_path(10, 0.05)

# max length
N = graph.vertices
K = N - 1

P = -10000

#binaries = range(floor(log2(K)) + 1)

tail_graph = make_tail_graph(graph, K)

# vertex_edges = make_vertex_edges(graph)
tail_vertex_edges = make_vertex_edges(tail_graph)

out_flow_edges = make_out_flow_edges(tail_graph)
in_flow_edges = make_in_flow_edges(tail_graph)


print("Original graph: ", graph.edges)
print("Graph with tail: ", tail_graph.edges)
print()

print("Vertex : Edges connected to vertex")
for i in range(len(tail_vertex_edges)):
    print(f"{i} : {tail_vertex_edges[i]}")


Original graph:  [(0, 1), (1, 2), (0, 2)]
Graph with tail:  [(0, 1), (1, 2), (0, 2), (0, 3), (1, 3), (2, 3), (3, 4)]

Vertex : Edges connected to vertex
0 : [0, 2, 3]
1 : [0, 1, 4]
2 : [1, 2, 5]
3 : [3, 4, 5, 6]
4 : [6]


In [40]:


vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY')
edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY')
initial_edge_vars = Array.create('init_edge', graph.vertices, vartype='BINARY')
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') 


#flow_vars = Array.create('flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY') #a variables
#contra_flow_vars = Array.create('contra_flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY') #b variables

# flow_vars_sums = [0 for _ in tail_graph.edges]
# contra_flow_vars_sums = [0 for _ in tail_graph.edges]

# for n in binaries:
# 	for i in range(len(tail_graph.edges)):
# 		flow_vars_sums[i] += 2**n * flow_vars[n][i]
# 		contra_flow_vars_sums[i] += 2**n * contra_flow_vars[n][i]


flow_vars = [LogEncInteger(f"flow_vars[{i}]", (0,K + 10)) for i in range(len(tail_graph.edges))]
initial_flow_vars = [LogEncInteger(f"initial_flow_vars[{i}]", (0,K + 10)) for i in range(graph.vertices)] #assume ingoing
terminal_flow_vars = [LogEncInteger(f"terminal_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume outgoing

contra_flow_vars = [LogEncInteger(f"contra_flow_vars[{i}]", (0,K + 10)) for i in range(len(tail_graph.edges))]
initial_contra_flow_vars = [LogEncInteger(f"initial_contra_flow_vars[{i}]", (0,K + 10)) for i in range(graph.vertices)] #assume outgoing
terminal_contra_flow_vars = [LogEncInteger(f"terminal_contra_flow_vars[{i}]", (0,K + 10)) for i in range(tail_graph.vertices)] #assume ingoing


#(1)
candy_exp = sum(edge_vars[i] for i in range(len(graph.edges)))

#(2)
initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2

#(3)
terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(tail_graph.vertices)))**2

#(4)
fake_terminal_node_exp_1 = P * (vertex_vars[graph.vertices] - sum(edge_vars[i] for i in range( len(graph.edges), len(graph.edges) + graph.vertices)))**2

print(fake_terminal_node_exp_1)

fake_terminal_node_exp_2 = P * (1 - vertex_vars[graph.vertices]) * vertex_vars[graph.vertices + 1]

#(5) and (6) can be together
real_node_deg_2_exp = 0
for i in range(tail_graph.vertices):
	temp1 = sum(edge_vars[tail_vertex_edges[i][j]] for j in range(len(tail_vertex_edges[i])))
	
	temp1 += terminal_edge_vars[i]

	if i < graph.vertices:
		temp1 += initial_edge_vars[i]

	# temp = sum(edge_vars[vertex_edges[i][j]] for j in range(len(vertex_edges[i])))
	# temp += initial_edge_vars[i] + terminal_edge_vars[i] + edge_vars[graph.vertices + i]

	print(temp1)
	# print("------")
	# print(temp)
	print()

	real_node_deg_2_exp += P * (2 * vertex_vars[i] - temp1)**2

#(6)
# tail_node_deg_2_exp = 0
# k = 0
# for i in range(graph.vertices + 1, tail_graph.vertices):
# 	tail_node_deg_2_exp += (2 * vertex_vars[i] - edge_vars[len(graph.edges) + k] - edge_vars[len(graph.edges) + k + 1] - terminal_edge_vars[i])**2
# 	#print((2 * vertex_vars[i] - edge_vars[len(graph.edges) + k] - edge_vars[len(graph.edges) + k + 1] - terminal_edge_vars[i])**2)
# tail_node_deg_2_exp = P * tail_node_deg_2_exp

#print(tail_node_deg_2_exp)

#(7) Paring the flow and contra flow
pairing_flow_exp = P * sum([((K + 3) * edge_vars[i] - flow_vars[i] - contra_flow_vars[i])**2 for i in range(len(edge_vars))])

pairing_flow_exp += P * sum([((K + 3) * initial_edge_vars[i] - initial_flow_vars[i] - initial_contra_flow_vars[i])**2 for i in range(len(initial_edge_vars))])

pairing_flow_exp += P * sum([((K + 3) * terminal_edge_vars[i] - terminal_flow_vars[i] - terminal_contra_flow_vars[i])**2 for i in range(len(terminal_edge_vars))])
	
#(7.2) setting initial and terminal flow values
set_initial_flow_exp = P * (1 - sum([a for a in initial_flow_vars]))**2
set_terminal_flow_exp = P * (1 - sum(b for b in terminal_contra_flow_vars))**2

#(7,3) setting the out-flow for all vertices that are used
set_out_flow_exp = 0
for i in range(tail_graph.vertices):
	temp_exp = (K+2) * vertex_vars[i]
	if i < graph.vertices:
		temp_exp += -initial_flow_vars[i]
	temp_exp += -terminal_contra_flow_vars[i]

	temp_exp += -sum([contra_flow_vars[j] for j in out_flow_edges[i]])
	temp_exp += -sum([flow_vars[j] for j in in_flow_edges[i]])

	temp_exp = temp_exp**2

	set_out_flow_exp += P * temp_exp	

	


#(7) labeling edges stuff
	

# skip (8) BAD

#(9)
# incomming_flow_exp = 0
# for i in range(tail_graph.vertices):
# 	temp_exp = (2 * K + 6) * vertex_vars[i]

# 	for j in tail_vertex_edges[i]:
# 		temp_exp -= flow_vars_sums[j]

# 		for k in tail_vertex_edges[i]:
# 			if k != j:
# 				temp_exp -= contra_flow_vars_sums[k]
	
# 	incomming_flow_exp += temp_exp**2
# incomming_flow_exp = P * incomming_flow_exp


(-10000.000000 * ((Binary('vertex[3]') + (-1.000000 * (Binary('edge[5]') + Binary('edge[4]') + 0.000000 + Binary('edge[3]')))) * (Binary('vertex[3]') + (-1.000000 * (Binary('edge[5]') + Binary('edge[4]') + 0.000000 + Binary('edge[3]'))))))
(Binary('init_edge[0]') + Binary('term_edge[0]') + Binary('edge[3]') + Binary('edge[2]') + 0.000000 + Binary('edge[0]'))

(Binary('init_edge[1]') + Binary('term_edge[1]') + Binary('edge[4]') + Binary('edge[1]') + 0.000000 + Binary('edge[0]'))

(Binary('init_edge[2]') + Binary('term_edge[2]') + Binary('edge[5]') + Binary('edge[2]') + 0.000000 + Binary('edge[1]'))

(Binary('term_edge[3]') + Binary('edge[6]') + Binary('edge[5]') + Binary('edge[4]') + 0.000000 + Binary('edge[3]'))

(Binary('term_edge[4]') + 0.000000 + Binary('edge[6]'))



In [41]:
"""
The variables in the block contain solutions for the longest paths in example graphs.
"""

"""
Solution for graph: 
vertices: 0, 1, 2
edges: (0,1), (1,2)
"""
line_sample = {'edge[2]': 0, 'contra_flow_vars[0][0]': 1, 'edge[3]': 0, 'contra_flow_vars[0][1]': 1, 'edge[0]': 1, 'contra_flow_vars[0][2]': 0, 'edge[1]': 1, 'contra_flow_vars[0][3]': 0, 'initial_flow_vars[1][3]': 0, 'flow_vars[0][3]': 0, 'contra_flow_vars[2][1]': 0, 'contra_flow_vars[1][0]': 0, 'initial_flow_vars[1][2]': 0, 'flow_vars[0][2]': 0, 'contra_flow_vars[2][0]': 0, 'contra_flow_vars[1][1]': 1, 'initial_flow_vars[1][1]': 0, 'flow_vars[0][1]': 1, 'contra_flow_vars[2][3]': 0, 'contra_flow_vars[1][2]': 0, 'initial_flow_vars[1][0]': 0, 'flow_vars[0][0]': 0, 'contra_flow_vars[2][2]': 0, 'contra_flow_vars[1][3]': 0, 'contra_flow_vars[3][0]': 0, 'contra_flow_vars[3][1]': 0, 'contra_flow_vars[3][2]': 0, 'contra_flow_vars[3][3]': 0, 'terminal_flow_vars[0][1]': 0, 'contra_flow_vars[4][0]': 0, 'terminal_flow_vars[0][0]': 0, 'contra_flow_vars[4][1]': 0, 'edge[4]': 0, 'terminal_flow_vars[0][3]': 0, 'contra_flow_vars[4][2]': 0, 'edge[5]': 0, 'terminal_flow_vars[0][2]': 0, 'contra_flow_vars[4][3]': 0, 'init_edge[2]': 0, 'terminal_flow_vars[3][3]': 0, 'initial_flow_vars[2][2]': 0, 'flow_vars[3][2]': 0, 'contra_flow_vars[5][0]': 0, 'terminal_flow_vars[3][2]': 0, 'initial_flow_vars[2][3]': 0, 'flow_vars[3][3]': 0, 'contra_flow_vars[5][1]': 0, 'init_edge[0]': 1, 'terminal_flow_vars[3][1]': 0, 'initial_flow_vars[2][0]': 0, 'flow_vars[3][0]': 0, 'contra_flow_vars[5][2]': 0, 'init_edge[1]': 0, 'terminal_flow_vars[3][0]': 0, 'initial_flow_vars[2][1]': 0, 'flow_vars[3][1]': 0, 'contra_flow_vars[5][3]': 0, 'flow_vars[2][1]': 0, 'initial_flow_vars[0][0]': 1, 'flow_vars[1][0]': 1,'flow_vars[2][0]': 0, 'initial_flow_vars[0][1]': 0, 'flow_vars[1][1]': 1, 'flow_vars[2][3]': 0, 'initial_flow_vars[0][2]': 0, 'flow_vars[1][2]': 0, 'flow_vars[2][2]': 0, 'initial_flow_vars[0][3]': 0, 'flow_vars[1][3]': 0, 'flow_vars[4][0]': 0, 'flow_vars[4][1]': 0, 'flow_vars[4][2]': 0, 'flow_vars[4][3]': 0, 'flow_vars[5][0]': 0,'flow_vars[5][1]': 0, 'flow_vars[5][2]': 0, 'flow_vars[5][3]': 0, 'initial_contra_flow_vars[0][0]': 0, 'initial_contra_flow_vars[0][1]': 0, 'initial_contra_flow_vars[0][2]': 1, 'initial_contra_flow_vars[0][3]': 0, 'initial_contra_flow_vars[1][0]': 0, 'initial_contra_flow_vars[1][1]': 0, 'initial_contra_flow_vars[1][2]': 0, 'initial_contra_flow_vars[1][3]': 0, 'initial_contra_flow_vars[2][0]': 0, 'initial_contra_flow_vars[2][1]': 0, 'initial_contra_flow_vars[2][2]': 0, 'initial_contra_flow_vars[2][3]': 0, 'term_edge[0]': 0, 'term_edge[1]': 0, 'term_edge[2]': 1, 'term_edge[3]': 0, 'term_edge[4]': 0, 'terminal_contra_flow_vars[0][0]': 0, 'terminal_contra_flow_vars[0][1]': 0, 'terminal_contra_flow_vars[0][2]': 0, 'terminal_contra_flow_vars[0][3]': 0, 'terminal_contra_flow_vars[1][0]': 0, 'terminal_contra_flow_vars[1][1]': 0, 'terminal_contra_flow_vars[1][2]': 0, 'terminal_contra_flow_vars[1][3]': 0, 'terminal_contra_flow_vars[2][0]': 1, 'terminal_contra_flow_vars[2][1]': 0, 'terminal_contra_flow_vars[2][2]': 0, 'terminal_contra_flow_vars[2][3]': 0, 'terminal_contra_flow_vars[3][0]': 0, 'terminal_contra_flow_vars[3][1]': 0, 'terminal_contra_flow_vars[3][2]': 0, 'terminal_contra_flow_vars[3][3]': 0, 'terminal_contra_flow_vars[4][0]': 0, 'terminal_contra_flow_vars[4][1]': 0, 'terminal_contra_flow_vars[4][2]': 0, 'terminal_contra_flow_vars[4][3]': 0, 'terminal_flow_vars[1][0]': 0, 'terminal_flow_vars[1][1]': 0, 'terminal_flow_vars[1][2]': 0, 'terminal_flow_vars[1][3]': 0, 'terminal_flow_vars[2][0]': 0, 'terminal_flow_vars[2][1]': 0, 'terminal_flow_vars[2][2]': 1, 'terminal_flow_vars[2][3]': 0, 'terminal_flow_vars[4][0]': 0, 'terminal_flow_vars[4][1]': 0, 'terminal_flow_vars[4][2]': 0, 'terminal_flow_vars[4][3]': 0, 'vertex[0]': 1, 'vertex[1]': 1, 'vertex[2]': 1, 'vertex[3]': 0, 'vertex[4]': 0}

"""
Solution for graph: 
vertices: 0, 1, 2
edges: (0,1), (1,2), (0,2)
"""
triangle_sample = {'edge[2]': 0, 'contra_flow_vars[0][0]': 0, 'edge[3]': 0, 'contra_flow_vars[0][1]': 0, 'edge[0]': 0, 'contra_flow_vars[0][2]': 0, 'edge[1]': 1, 'contra_flow_vars[0][3]': 0, 'flow_vars[0][3]': 0, 'initial_flow_vars[1][3]': 0, 'contra_flow_vars[2][1]': 0, 'contra_flow_vars[1][0]': 0, 'flow_vars[0][2]': 0, 'initial_flow_vars[1][2]': 0, 'contra_flow_vars[2][0]': 0, 'contra_flow_vars[1][1]': 1, 'flow_vars[0][1]': 0, 'initial_flow_vars[1][1]': 0, 'contra_flow_vars[2][3]': 0, 'contra_flow_vars[1][2]': 0, 'flow_vars[0][0]': 0, 'initial_flow_vars[1][0]': 0, 'contra_flow_vars[2][2]': 0, 'contra_flow_vars[1][3]': 0, 'contra_flow_vars[3][0]': 0, 'contra_flow_vars[3][1]': 0, 'contra_flow_vars[3][2]': 0, 'contra_flow_vars[3][3]': 0, 'edge[6]': 0, 'contra_flow_vars[4][0]': 0, 'terminal_flow_vars[0][1]': 0, 'contra_flow_vars[4][1]': 1, 'terminal_flow_vars[0][0]': 0, 'edge[4]': 1, 'contra_flow_vars[4][2]': 0, 'terminal_flow_vars[0][3]': 0, 'edge[5]': 0, 'contra_flow_vars[4][3]': 0, 'terminal_flow_vars[0][2]': 0, 'flow_vars[3][2]': 0, 'initial_flow_vars[2][2]': 0, 'terminal_flow_vars[3][3]': 0, 'contra_flow_vars[6][1]': 0, 'contra_flow_vars[5][0]': 0, 'flow_vars[3][3]': 0, 'initial_flow_vars[2][3]': 0, 'terminal_flow_vars[3][2]': 1, 'contra_flow_vars[6][0]': 0, 'contra_flow_vars[5][1]': 0, 'init_edge[0]': 0, 'flow_vars[3][0]': 0, 'initial_flow_vars[2][0]': 1, 'terminal_flow_vars[3][1]': 0, 'contra_flow_vars[6][3]': 0, 'contra_flow_vars[5][2]': 0, 'flow_vars[3][1]': 0, 'initial_flow_vars[2][1]': 0, 'terminal_flow_vars[3][0]': 0, 'contra_flow_vars[6][2]': 0, 'contra_flow_vars[5][3]': 0, 'flow_vars[2][1]': 0, 'flow_vars[1][0]': 1, 'initial_flow_vars[0][0]': 0, 'flow_vars[2][0]': 0, 'flow_vars[1][1]': 1, 'initial_flow_vars[0][1]': 0, 'flow_vars[2][3]': 0, 'flow_vars[1][2]': 0, 'initial_flow_vars[0][2]': 0, 'flow_vars[2][2]': 0, 'flow_vars[1][3]': 0, 'initial_flow_vars[0][3]': 0, 'flow_vars[4][0]': 1, 'flow_vars[4][1]': 1, 'flow_vars[4][2]': 0, 'flow_vars[4][3]': 0, 'flow_vars[6][1]': 0, 'flow_vars[5][0]': 0, 'flow_vars[6][0]': 0, 'flow_vars[5][1]': 0, 'flow_vars[6][3]': 0, 'flow_vars[5][2]': 0, 'flow_vars[6][2]': 0, 'flow_vars[5][3]': 0, 'init_edge[1]': 0, 'init_edge[2]': 1, 'initial_contra_flow_vars[0][0]': 0, 'initial_contra_flow_vars[0][1]': 0, 'initial_contra_flow_vars[0][2]': 0, 'initial_contra_flow_vars[0][3]': 0, 'initial_contra_flow_vars[1][0]': 0, 'initial_contra_flow_vars[1][1]': 0, 'initial_contra_flow_vars[1][2]': 0, 'initial_contra_flow_vars[1][3]': 0, 'initial_contra_flow_vars[2][0]': 0, 'initial_contra_flow_vars[2][1]': 0, 'initial_contra_flow_vars[2][2]': 1, 'initial_contra_flow_vars[2][3]': 0, 'term_edge[0]': 0, 'term_edge[1]': 0, 'term_edge[2]': 0, 'term_edge[3]': 1, 'term_edge[4]': 0, 'terminal_contra_flow_vars[0][0]': 0, 'terminal_contra_flow_vars[0][1]': 0, 'terminal_contra_flow_vars[0][2]': 0, 'terminal_contra_flow_vars[0][3]': 0, 'terminal_contra_flow_vars[1][0]': 0, 'terminal_contra_flow_vars[1][1]': 0, 'terminal_contra_flow_vars[1][2]': 0, 'terminal_contra_flow_vars[1][3]': 0, 'terminal_contra_flow_vars[2][0]': 0, 'terminal_contra_flow_vars[2][1]': 0, 'terminal_contra_flow_vars[2][2]': 0, 'terminal_contra_flow_vars[2][3]': 0, 'terminal_contra_flow_vars[3][0]': 1, 'terminal_contra_flow_vars[3][1]': 0, 'terminal_contra_flow_vars[3][2]': 0, 'terminal_contra_flow_vars[3][3]': 0, 'terminal_contra_flow_vars[4][0]': 0, 'terminal_contra_flow_vars[4][1]': 0, 'terminal_contra_flow_vars[4][2]': 0, 'terminal_contra_flow_vars[4][3]': 0, 'terminal_flow_vars[1][0]': 0, 'terminal_flow_vars[1][1]': 0, 'terminal_flow_vars[1][2]': 0, 'terminal_flow_vars[1][3]': 0, 'terminal_flow_vars[2][0]': 0, 'terminal_flow_vars[2][1]': 0, 'terminal_flow_vars[2][2]': 0, 'terminal_flow_vars[2][3]': 0, 'terminal_flow_vars[4][0]': 0, 'terminal_flow_vars[4][1]': 0, 'terminal_flow_vars[4][2]': 0, 'terminal_flow_vars[4][3]': 0, 'vertex[0]': 0, 'vertex[1]': 1, 'vertex[2]': 1, 'vertex[3]': 1, 'vertex[4]': 0}


In [42]:
total_exp = candy_exp + initial_node_exp + terminal_node_exp + fake_terminal_node_exp_1 + fake_terminal_node_exp_2 + real_node_deg_2_exp + pairing_flow_exp + set_initial_flow_exp + set_terminal_flow_exp + set_out_flow_exp
# + tail_node_deg_2_exp # (6) is included in (5).

#total_exp = candy_exp +  initial_node_exp + terminal_node_exp + fake_terminal_node_exp_1 + fake_terminal_node_exp_2 + real_node_deg_2_exp + pairing_flow_exp

exp = -total_exp

model = exp.compile()
bqm = model.to_bqm()

sa = neal.SimulatedAnnealingSampler()

#Use the upper line to input a solution
#sampleset = sa.sample(bqm, num_reads=1, num_sweeps=0, initial_states= triangle_sample)
sampleset = sa.sample(bqm, num_reads=10000, num_sweeps=1000)


decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)
print(best_sample.energy)

print(best_sample.sample)
print("Hi---------------")

print("Vertices:")
for i in range(tail_graph.vertices):
	print(f"vertex {i} : ", best_sample.sample[f"vertex[{i}]"])

print()
print("Edges:")
for i in range(len(tail_graph.edges)):
	print(f"Edge {i} : ", best_sample.sample[f"edge[{i}]"])

# print()
# print("Flow vars:")
# for i in range(len(tail_graph.edges)):
# 	for j in range(ceil(log2(K + 10))):
# 		print(f"Flow var {i} {j} : ", best_sample.sample[f"flow_vars[{i}][{j}]"])

print()

print("Initial flow vars:")
for i in range(graph.vertices):	
	print(f"Initial flow var {i}: ", best_sample.subh[f"initial_flow_vars[{i}]"])

print("Flow vars:")
for i in range(len(tail_graph.edges)):	
	print(f"Flow var {i}: ", best_sample.subh[f"flow_vars[{i}]"])

print("Terminal flow vars:")
for i in range(tail_graph.vertices):
	print(f"Terminal flow var {i}: ", best_sample.subh[f"terminal_flow_vars[{i}]"])


print()

print("Contra initial flow vars:")
for i in range(graph.vertices):	
	print(f"Initial contra flow var {i}: ", best_sample.subh[f"initial_contra_flow_vars[{i}]"])

print("Contra flow vars:")
for i in range(len(tail_graph.edges)):	
	print(f"Contra flow var {i}: ", best_sample.subh[f"contra_flow_vars[{i}]"])

print("Contra terminal flow vars:")
for i in range(tail_graph.vertices):	
	print(f"Terminal contra flow var {i}: ", best_sample.subh[f"terminal_contra_flow_vars[{i}]"])

print(tail_graph.edges)